# FE009 - More OpenBabel Features
- Add more neighbors
- Add weight of neightbor atoms

In [1]:
!pip install modin

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pandas==0.24.2

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import modin.pandas as pd

2019-06-26 12:45:02,650	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-06-26 12:45:02,652	INFO node.py:497 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-26_12-45-02_651565_2801/logs.
2019-06-26 12:45:02,760	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:10924 to respond...
2019-06-26 12:45:02,886	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:15459 to respond...
2019-06-26 12:45:02,888	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-06-26 12:45:02,917	INFO node.py:511 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-26_12-45-02_651565_2801/logs.
2019-06-26 12:45:02,919	WARNING services.py:1318 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may

In [4]:
# import pandas as pd
import matplotlib.pylab as plt

In [13]:
import dask.dataframe as dd

In [5]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [6]:
%%time
angle_details = pd.read_csv('../data/openbabel/angle_details.csv')
atom_details = pd.read_csv('../data/openbabel/atom_details.csv')
bond_details = pd.read_csv('../data/openbabel/bond_details.csv')
closest = pd.read_csv('../data/openbabel/closest.csv')
distances = pd.read_csv('../data/openbabel/distances.csv')
molecule_details = pd.read_csv('../data/openbabel/molecule_details.csv')
torsion_details = pd.read_csv('../data/openbabel/torsion_details.csv')

CPU times: user 686 ms, sys: 153 ms, total: 840 ms
Wall time: 4.96 s


In [7]:
tt = pd.concat([train, test], axis=0, sort=False)

In [8]:
tt.shape

(7163689, 6)

In [9]:
tt.head()

,atom_index_0,atom_index_1,id,molecule_name,scalar_coupling_constant,type
0,1,0,0,dsgdb9nsd_000001,84.8076,1JHC
1,1,2,1,dsgdb9nsd_000001,-11.2570,2JHH
2,1,3,2,dsgdb9nsd_000001,-11.2548,2JHH
3,1,4,3,dsgdb9nsd_000001,-11.2543,2JHH
4,2,0,4,dsgdb9nsd_000001,84.8074,1JHC


# Atom Details

In [10]:
atom_details.head()

,molecule_name,atom_idx,atomic_mass,atomic_number,exact_mass,valence,spin_multiplicity
0,dsgdb9nsd_033805,0,14.0067,7,14.003074,1,0
1,dsgdb9nsd_033805,1,12.0107,6,12.000000,2,0
2,dsgdb9nsd_033805,2,12.0107,6,12.000000,4,0
3,dsgdb9nsd_033805,3,12.0107,6,12.000000,4,0
4,dsgdb9nsd_033805,4,12.0107,6,12.000000,4,0


In [11]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','atom_index_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'atom0_atomic_number',
                     'atomic_mass':'atom0_atomic_mass',
                     'valence':'atom0_valence',
                     'spin_multiplicity':'atom0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','atom_index_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'atom1_atomic_number',
                     'atomic_mass':'atom1_atomic_mass',
                     'valence':'atom1_valence',
                     'spin_multiplicity':'atom1_spin_multiplicity'})
tt.shape

To request implementation, send an email to feature_requests@modin.org.


CPU times: user 12.5 s, sys: 3.5 s, total: 16 s
Wall time: 20.1 s


# Angles

In [14]:
%%time
tt = pd.merge(tt,
         angle_details.groupby(['molecule_name','left_atom_idx','middle_atom_idx']).mean() \
              .reset_index(drop=False),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','middle_atom_idx'],
         how='left') \
    .drop(['left_atom_idx','middle_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'angle':'left_middle_average_angle'})

CPU times: user 12min 14s, sys: 17.3 s, total: 12min 31s
Wall time: 12min 31s


In [15]:
%%time
tt = pd.merge(tt,
         angle_details.groupby(['molecule_name','right_atom_idx','middle_atom_idx']).mean() \
              .reset_index(drop=False),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','right_atom_idx','middle_atom_idx'],
         how='left') \
    .drop(['left_atom_idx','middle_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'angle':'right_middle_average_angle'})

CPU times: user 12min 33s, sys: 18.7 s, total: 12min 51s
Wall time: 12min 55s


In [16]:
%%time
tt = pd.merge(tt,
         distances,
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','left_atom_idx','right_atom_idx'],
         how='left')\
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

RayTaskError: [36mray_worker[39m (pid=3100, host=092c89ff919c)
  File "/opt/conda/lib/python3.6/site-packages/ray/memory_monitor.py", line 77, in raise_if_low_memory
    self.error_threshold))
ray.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node 092c89ff919c is used (65.35 / 67.48 GB). The top 5 memory consumers are:

PID	MEM	COMMAND
4291	37.24GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e58544bc-8f
4206	12.18GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-80f8b511-28
2801	4.89GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-5c36ebca-24
3100	1.15GB	/opt/conda/bin/python /opt/conda/lib/python3.6/site-packages/ray/workers/default_worker.py --node-ip
3093	0.94GB	/opt/conda/bin/python /opt/conda/lib/python3.6/site-packages/ray/workers/default_worker.py --node-ip

In addition, ~0.13 GB of shared memory is currently being used by the R

# Closest Neighbors

In [17]:
# Fix bug with the indexes being off by one
if closest['closest'].min() == 1:
    closest['atom_index'] = closest['atom_index'] - 1
    for col in closest.columns:
        if 'close' in col:
            closest[col] = closest[col] - 1

In [18]:
%%time
tt = pd.merge(tt,
         closest[['molecule_name','atom_index','closest','2nd_closest','3rd_closest','4th_closest',
                  '5th_closest', '6th_closest','7th_closest','8th_closest','9th_closest','10th_closest']],
        left_on=['molecule_name','atom_index_0'],
        right_on=['molecule_name','atom_index'],
        how='left') \
    .drop(['atom_index'], axis=1) \
    .rename(columns={'closest':'closest_to_0',
                     '2nd_closest':'2nd_closest_to_0',
                     '3rd_closest':'3rd_closest_to_0',
                     '4th_closest':'4th_closest_to_0',
                     '5th_closest':'5th_closest_to_0',
                     '6th_closest':'6th_closest_to_0',
                     '7th_closest':'7th_closest_to_0',
                     '8th_closest':'8th_closest_to_0',
                     '9th_closest':'9th_closest_to_0',
                     '10th_closest':'10th_closest_to_0'})

RayTaskError: [36mray_worker[39m (pid=3100, host=092c89ff919c)
  File "/opt/conda/lib/python3.6/site-packages/ray/memory_monitor.py", line 77, in raise_if_low_memory
    self.error_threshold))
ray.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node 092c89ff919c is used (65.35 / 67.48 GB). The top 5 memory consumers are:

PID	MEM	COMMAND
4291	37.24GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e58544bc-8f
4206	12.18GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-80f8b511-28
2801	4.89GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-5c36ebca-24
3100	1.15GB	/opt/conda/bin/python /opt/conda/lib/python3.6/site-packages/ray/workers/default_worker.py --node-ip
3093	0.94GB	/opt/conda/bin/python /opt/conda/lib/python3.6/site-packages/ray/workers/default_worker.py --node-ip

In addition, ~0.13 GB of shared memory is currently being used by the R

In [19]:
%%time
tt = pd.merge(tt,
         closest[['molecule_name','atom_index','closest','2nd_closest','3rd_closest','4th_closest',
                  '5th_closest', '6th_closest','7th_closest','8th_closest','9th_closest','10th_closest']],
        left_on=['molecule_name','atom_index_1'],
        right_on=['molecule_name','atom_index'],
        how='left') \
    .drop(['atom_index'], axis=1) \
    .rename(columns={'closest':'closest_to_1',
                     '2nd_closest':'2nd_closest_to_1',
                     '3rd_closest':'3rd_closest_to_1',
                     '4th_closest':'4th_closest_to_1',
                     '5th_closest':'5th_closest_to_1',
                     '6th_closest':'6th_closest_to_1',
                     '7th_closest':'7th_closest_to_1',
                     '8th_closest':'8th_closest_to_1',
                     '9th_closest':'9th_closest_to_1',
                     '10th_closest':'10th_closest_to_1'})

RayTaskError: [36mray_worker[39m (pid=3100, host=092c89ff919c)
  File "/opt/conda/lib/python3.6/site-packages/ray/memory_monitor.py", line 77, in raise_if_low_memory
    self.error_threshold))
ray.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node 092c89ff919c is used (65.35 / 67.48 GB). The top 5 memory consumers are:

PID	MEM	COMMAND
4291	37.24GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-e58544bc-8f
4206	12.18GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-80f8b511-28
2801	4.89GB	/opt/conda/bin/python -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-5c36ebca-24
3100	1.15GB	/opt/conda/bin/python /opt/conda/lib/python3.6/site-packages/ray/workers/default_worker.py --node-ip
3093	0.94GB	/opt/conda/bin/python /opt/conda/lib/python3.6/site-packages/ray/workers/default_worker.py --node-ip

In addition, ~0.13 GB of shared memory is currently being used by the R

In [20]:
tt.shape

(7163689, 18)

In [21]:
tt['is_closest_pair'] = tt['closest_to_0'] == tt['atom_index_1']
tt.shape

KeyError: 'closest_to_0'

# Distance to neighbors

## To atom0

In [ ]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

In [ ]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','2nd_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_2nd_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

In [ ]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','3rd_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_3rd_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','4th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_4th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','5th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_5th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','6th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_6th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','7th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_7th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)

In [ ]:
%%time
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','8th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_8th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','9th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_9th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt = pd.merge(tt,
        distances,
        left_on=['molecule_name','atom_index_0','10th_closest_to_0'],
        right_on=['molecule_name','left_atom_idx','right_atom_idx'],
        suffixes=('','_10th_closest_to_0'),
             how='left') \
    .drop(['left_atom_idx','right_atom_idx'], axis=1)
tt.shape

## To atom1

In [ ]:
%%time
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', 'closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '2nd_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_2nd_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '3rd_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_3rd_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '4th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_4th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1

In [ ]:
%%time
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '5th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_5th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '6th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_6th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '7th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_7th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)

In [ ]:
%%time
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '8th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_8th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '9th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_9th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt = pd.merge(tt,
              distances,
              left_on=['molecule_name', 'atom_index_1', '10th_closest_to_1'],
              right_on=['molecule_name', 'left_atom_idx', 'right_atom_idx'],
              suffixes=('', '_10th_closest_to_1'),
              how='left') \
    .drop(['left_atom_idx', 'right_atom_idx'], axis=1)
tt.shape

# Atom Details of Neighbors

In [ ]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'closest_to_0_atomic_number',
                     'atomic_mass':'closest_to_0_atomic_mass',
                     'valence':'closest_to_0_valence',
                     'spin_multiplicity':'closest_to_0_spin_multiplicity'})
tt.shape

In [ ]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','2nd_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'2nd_closest_to_0_atomic_number',
                     'atomic_mass':'2nd_closest_to_0_atomic_mass',
                     'valence':'2nd_closest_to_0_valence',
                     'spin_multiplicity':'2nd_closest_to_0_spin_multiplicity'})
tt.shape

In [ ]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','3rd_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'3rd_closest_to_0_atomic_number',
                     'atomic_mass':'3rd_closest_to_0_atomic_mass',
                     'valence':'3rd_closest_to_0_valence',
                     'spin_multiplicity':'3rd_closest_to_0_spin_multiplicity'})
tt.shape

In [ ]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','4th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'4th_closest_to_0_atomic_number',
                     'atomic_mass':'4th_closest_to_0_atomic_mass',
                     'valence':'4th_closest_to_0_valence',
                     'spin_multiplicity':'4th_closest_to_0_spin_multiplicity'})
tt.shape

In [ ]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','5th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'5th_closest_to_0_atomic_number',
                     'atomic_mass':'5th_closest_to_0_atomic_mass',
                     'valence':'5th_closest_to_0_valence',
                     'spin_multiplicity':'5th_closest_to_0_spin_multiplicity'})

In [ ]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','6th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'6th_closest_to_0_atomic_number',
                     'atomic_mass':'6th_closest_to_0_atomic_mass',
                     'valence':'6th_closest_to_0_valence',
                     'spin_multiplicity':'6th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','7th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'7th_closest_to_0_atomic_number',
                     'atomic_mass':'7th_closest_to_0_atomic_mass',
                     'valence':'7th_closest_to_0_valence',
                     'spin_multiplicity':'7th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','8th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'8th_closest_to_0_atomic_number',
                     'atomic_mass':'8th_closest_to_0_atomic_mass',
                     'valence':'8th_closest_to_0_valence',
                     'spin_multiplicity':'8th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','9th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'9th_closest_to_0_atomic_number',
                     'atomic_mass':'9th_closest_to_0_atomic_mass',
                     'valence':'9th_closest_to_0_valence',
                     'spin_multiplicity':'9th_closest_to_0_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','10th_closest_to_0'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'10th_closest_to_0_atomic_number',
                     'atomic_mass':'10th_closest_to_0_atomic_mass',
                     'valence':'10th_closest_to_0_valence',
                     'spin_multiplicity':'10th_closest_to_0_spin_multiplicity'})

In [ ]:
%%time
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'closest_to_1_atomic_number',
                     'atomic_mass':'closest_to_1_atomic_mass',
                     'valence':'closest_to_1_valence',
                     'spin_multiplicity':'closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','2nd_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'2nd_closest_to_1_atomic_number',
                     'atomic_mass':'2nd_closest_to_1_atomic_mass',
                     'valence':'2nd_closest_to_1_valence',
                     'spin_multiplicity':'2nd_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','3rd_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'3rd_closest_to_1_atomic_number',
                     'atomic_mass':'3rd_closest_to_1_atomic_mass',
                     'valence':'3rd_closest_to_1_valence',
                     'spin_multiplicity':'3rd_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','4th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'4th_closest_to_1_atomic_number',
                     'atomic_mass':'4th_closest_to_1_atomic_mass',
                     'valence':'4th_closest_to_1_valence',
                     'spin_multiplicity':'4th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','5th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'5th_closest_to_1_atomic_number',
                     'atomic_mass':'5th_closest_to_1_atomic_mass',
                     'valence':'5th_closest_to_1_valence',
                     'spin_multiplicity':'5th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','6th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'6th_closest_to_1_atomic_number',
                     'atomic_mass':'6th_closest_to_1_atomic_mass',
                     'valence':'6th_closest_to_1_valence',
                     'spin_multiplicity':'6th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','7th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'7th_closest_to_1_atomic_number',
                     'atomic_mass':'7th_closest_to_1_atomic_mass',
                     'valence':'7th_closest_to_1_valence',
                     'spin_multiplicity':'7th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','8th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'8th_closest_to_1_atomic_number',
                     'atomic_mass':'8th_closest_to_1_atomic_mass',
                     'valence':'8th_closest_to_1_valence',
                     'spin_multiplicity':'8th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','9th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'9th_closest_to_1_atomic_number',
                     'atomic_mass':'9th_closest_to_1_atomic_mass',
                     'valence':'9th_closest_to_1_valence',
                     'spin_multiplicity':'9th_closest_to_1_spin_multiplicity'})
tt = pd.merge(tt,
         atom_details,
         left_on=['molecule_name','10th_closest_to_1'],
         right_on=['molecule_name','atom_idx'],
         how='left') \
    .drop(['atom_idx'], axis=1) \
    .rename(columns={'atomic_number':'10th_closest_to_1_atomic_number',
                     'atomic_mass':'10th_closest_to_1_atomic_mass',
                     'valence':'10th_closest_to_1_valence',
                     'spin_multiplicity':'10th_closest_to_1_spin_multiplicity'})

## Torsion Details

In [ ]:
%%time
tt = pd.merge(tt,
        torsion_details.groupby(['molecule_name','2left_atom_idx','left_atom_idx'])['torsion_angle'] \
                  .agg(['mean','min','max','count'])\
                  .reset_index(),
        left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','left_atom_idx'],
         how='left') \
    .drop(['2left_atom_idx','left_atom_idx'], axis=1) \
    .rename(columns={'mean': 'tor_ang_2leftleft_mean',
                     'min': 'tor_ang_2leftleft_min',
                    'max': 'tor_ang_2leftleft_max',
                    'count': 'tor_ang_2leftleft_count'})
tt.shape

In [ ]:
%%time
tt = pd.merge(tt,
         torsion_details.groupby(['molecule_name','2left_atom_idx','right_atom_idx'])['torsion_angle'] \
                  .agg(['mean','min','max','count'])\
                  .reset_index(),
         left_on=['molecule_name','atom_index_0','atom_index_1'],
         right_on=['molecule_name','2left_atom_idx','right_atom_idx'],
         how='left') \
    .drop(['2left_atom_idx','right_atom_idx'], axis=1) \
    .rename(columns={'mean': 'tor_ang_2leftright_mean',
                     'min': 'tor_ang_2leftright_min',
                     'max': 'tor_ang_2leftright_max',
                     'count': 'tor_ang_2leftright_count'})
tt.shape

In [ ]:
molecule_details.head()

In [ ]:
%%time
tt = pd.merge(tt,
        molecule_details.drop('num_residues', axis=1),
         left_on=['molecule_name'],
         right_on=['molecule_name'],
         how='left')

In [ ]:
test.head()

In [ ]:
test_FE009 = tt.sort_values('id').loc[tt['scalar_coupling_constant'].isnull()].reset_index(drop=True)
train_FE009 = tt.sort_values('id').loc[~tt['scalar_coupling_constant'].isnull()].reset_index(drop=True)

In [ ]:
test_FE009.shape

In [ ]:
test.shape

In [ ]:
train_FE009.shape

In [ ]:
train.shape

In [ ]:
bool_cols = [col for col in train_FE009.columns if 'is_bond_' in col]
test_FE009[bool_cols] = test_FE009[bool_cols].fillna(False)
train_FE009[bool_cols] = train_FE009[bool_cols].fillna(False)

In [ ]:
test_FE009.to_parquet('../data/FE009_test.parquet')

In [ ]:
train_FE009.to_parquet('../data/FE009_train.parquet')

In [ ]:
[col for col in test_FE009.columns]